In [27]:
import re
import nltk
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from fast_ml.model_development import train_valid_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shivamarora/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [28]:
import numpy as np
import pandas as pd
from fast_ml.model_development import train_valid_test_split
from string import punctuation
import re
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn.naive_bayes as naive_bayes
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shivamarora/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shivamarora/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
columns_LIAR = ["ID","label","statement","subject","speaker","jobtitle","state","party","barely","false","half","mostly","pantsfire","context"]
df = pd.read_csv('../data/train.tsv', sep='\t', header= None, names=columns_LIAR)
df1 = pd.read_csv('../data/test.tsv', sep='\t', header= None, names=columns_LIAR)
df2 = pd.read_csv('../data/valid.tsv', sep='\t', header= None, names=columns_LIAR)

In [30]:
df = pd.concat([df, df1, df2], ignore_index=True)

In [31]:
df.shape

(12791, 14)

In [32]:
df = df.drop_duplicates(keep='last', ignore_index=True)
df = df.dropna()
df = df.reset_index(drop=True)

df['label'] = df['label'].map({'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true' : 3, 'mostly-true' : 4, 'true' : 5})

In [33]:
df.head()

,ID,label,statement,subject,speaker,jobtitle,state,party,barely,false,half,mostly,pantsfire,context
0,2635.json,1,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,3,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,4,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,12465.json,5,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
4,153.json,3,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."


In [34]:
df.shape

(8438, 14)

## Utility Functions

In [35]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in punctuation])
    return punctuationfree

In [36]:
def tokenization(text):
    return word_tokenize(str(text))

In [37]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopWords]
    return output

In [38]:
porter_stemmer = PorterStemmer()
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

In [39]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

## Preprocessing

In [40]:
def preprocess(df, col):
    
    #Puntuations Removal
    df[col] = df[col].apply(lambda x:remove_punctuation(x))
    
    #Numbers Removal
    remove_digits = str.maketrans('', '', digits)
    df[col] = df[col].apply(lambda x: x.translate(remove_digits))
    
    #lowering the text
    df[col] = df[col].apply(lambda x: x.lower())
    
    #tokenization
    df[col] = df[col].apply(lambda x: tokenization(x))
    
    #Removing stop words
    df[col] = df[col].apply(lambda x:remove_stopwords(x))
    
    #Lemmatization
    df[col] = df[col].apply(lambda x:lemmatizer(x))
    
    return df

In [41]:
enc = OneHotEncoder(handle_unknown='ignore')

In [42]:
enc_df = pd.DataFrame(enc.fit_transform(df[['label']]).toarray())
enc_df.shape

(8438, 6)

In [43]:
enc_df

,0,1,2,3,4,5
0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
8433,0.0,0.0,1.0,0.0,0.0,0.0
8434,1.0,0.0,0.0,0.0,0.0,0.0
8435,0.0,0.0,0.0,1.0,0.0,0.0
8436,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
df = preprocess(df, 'statement')

In [46]:
df.head()

,ID,label,statement,subject,speaker,jobtitle,state,party,barely,false,half,mostly,pantsfire,context
0,2635.json,1,"[say, annies, list, political, group, support,...",abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,3,"[decline, coal, start, started, natural, gas, ...","energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,4,"[hillary, clinton, agrees, john, mccain, votin...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,12465.json,5,"[chicago, bear, starting, quarterback, last, y...",education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
4,153.json,3,"[im, person, stage, worked, actively, last, ye...",ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."


In [54]:
df = preprocess(df, 'subject')
df = preprocess(df, 'speaker')
df = preprocess(df, 'jobtitle')
df = preprocess(df, 'state')
df = preprocess(df, 'party')
df = preprocess(df, 'context')


In [55]:
df = df.join(enc_df)

ValueError: columns overlap but no suffix specified: Index([0, 1, 2, 3, 4, 5], dtype='object')

In [56]:
df.head()

,ID,label,statement,subject,speaker,jobtitle,state,party,barely,false,half,mostly,pantsfire,context,0,1,2,3,4,5
0,2635.json,1,"[say, annies, list, political, group, support,...",[abortion],[dwaynebohac],"[state, representative]",[texas],[republican],0.0,1.0,0.0,0.0,0.0,[mailer],0.0,1.0,0.0,0.0,0.0,0.0
1,10540.json,3,"[decline, coal, start, started, natural, gas, ...",[energyhistoryjobaccomplishments],[scottsurovell],"[state, delegate]",[virginia],[democrat],0.0,0.0,1.0,1.0,0.0,"[floor, speech]",0.0,0.0,0.0,1.0,0.0,0.0
2,324.json,4,"[hillary, clinton, agrees, john, mccain, votin...",[foreignpolicy],[barackobama],[president],[illinois],[democrat],70.0,71.0,160.0,163.0,9.0,[denver],0.0,0.0,0.0,0.0,1.0,0.0
3,12465.json,5,"[chicago, bear, starting, quarterback, last, y...",[education],[robinvos],"[wisconsin, assembly, speaker]",[wisconsin],[republican],0.0,3.0,2.0,5.0,1.0,"[online, opinionpiece]",0.0,0.0,0.0,0.0,0.0,1.0
4,153.json,3,"[im, person, stage, worked, actively, last, ye...",[ethic],[barackobama],[president],[illinois],[democrat],70.0,71.0,160.0,163.0,9.0,"[democratic, debate, philadelphia, pa]",0.0,0.0,0.0,1.0,0.0,0.0


In [57]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'label', 
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)

In [58]:
X_train

,ID,statement,subject,speaker,jobtitle,state,party,barely,false,half,mostly,pantsfire,context,0,1,2,3,4,5
4585,1181.json,"[lie, response, president, obama, saying, heal...",[healthcareimmigration],[joewilson],[congressman],"[south, carolina]",[republican],0.0,1.0,0.0,0.0,0.0,"[audience, joint, session, congress]",0.0,1.0,0.0,0.0,0.0,0.0
2014,12085.json,"[able, provide, million, worth, property, tax,...",[statebudgettaxes],[dongaetz],"[state, senator]",[florida],[republican],1.0,0.0,3.0,1.0,0.0,"[florida, senate, floor, debate]",0.0,0.0,0.0,0.0,1.0,0.0
6999,11226.json,"[even, federal, income, tax, country]",[historytaxes],[jimwebb],[writer],[virginia],[democrat],1.0,0.0,1.0,5.0,0.0,"[web, post]",0.0,0.0,0.0,0.0,1.0,0.0
7786,8721.json,"[say, david, jolly, lobbied, governmentrun, he...",[candidatesbiographycampaignadvertising],[kathleenpeters],"[district, state, representative]",[florida],[republican],1.0,1.0,0.0,1.0,0.0,"[comment, press, conference]",0.0,1.0,0.0,0.0,0.0,0.0
2634,7266.json,"[total, added, nearly, private, sector, job, n...",[economyjobs],[chrischristie],"[governor, new, jersey]","[new, jersey]",[republican],10.0,17.0,27.0,19.0,8.0,"[state, state, address]",0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,2248.json,"[since, president, became, president, lost, mi...",[economyfederalbudget],[stevelatourette],"[former, u, representative, ohio, th, district]",[ohio],[republican],0.0,0.0,3.0,1.0,0.0,[speech],0.0,0.0,0.0,1.0,0.0,0.0
6558,297.json,"[reformed, welfare, others, tried]",[jobaccomplishments],[rudygiuliani],[attorney],"[new, york]",[republican],9.0,11.0,10.0,7.0,3.0,"[tv, ad]",0.0,1.0,0.0,0.0,0.0,0.0
3476,6519.json,"[mitt, romney, say, class, size, dont, matter]",[educationmessagemachine],[barackobama],[president],[illinois],[democrat],70.0,71.0,160.0,163.0,9.0,"[campaign, ad]",0.0,0.0,0.0,0.0,0.0,1.0
7724,6029.json,"[forty, year, ago, half, congressional, distri...",[bipartisanshiphistory],[johnbarrow],[congressman],[georgia],[democrat],0.0,0.0,1.0,1.0,0.0,[oped],0.0,0.0,0.0,1.0,0.0,0.0


In [59]:
y_train

4585    1
2014    4
6999    4
7786    1
2634    5
       ..
189     3
6558    1
3476    5
7724    3
6360    5
Name: label, Length: 6750, dtype: int64

In [65]:
def identity_tokenizer(text):
    return text
tfidf_vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words='english', lowercase=False, max_features=2000)
tfidf_vectorizer.fit(df['statement'])

TfidfVectorizer(lowercase=False, max_features=2000, stop_words='english',
                tokenizer=<function identity_tokenizer at 0x7fc924d0c940>)

In [66]:
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train['statement'])
tfidf_valid_vectors = tfidf_vectorizer.fit_transform(X_valid['statement'])
tfidf_test_vectors = tfidf_vectorizer.fit_transform(X_test['statement'])

In [67]:
tfidf_train_vectors

<6750x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 51165 stored elements in Compressed Sparse Row format>

In [68]:
tfidf_test_vectors

<844x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 7052 stored elements in Compressed Sparse Row format>

In [69]:
tfidf_valid_vectors

<844x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 6942 stored elements in Compressed Sparse Row format>

In [70]:
transformer = FeatureUnion([
                ('statement_tfidf', 
                  Pipeline([('extract_field',
                              FunctionTransformer(lambda x: x['statement'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('subject_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['subject'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('speaker_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['speaker'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('jobtitle_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['jobtitle'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('state_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['state'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('party_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['party'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())])),
                ('context_tfidf', 
                  Pipeline([('extract_field', 
                              FunctionTransformer(lambda x: x['context'], 
                                                  validate=False)),
                            ('tfidf', 
                              TfidfVectorizer())]))

])

In [71]:
tfidf_train_vectors = transformer.fit(X_train)
Xt = transformer.transform(X_train)

AttributeError: 'list' object has no attribute 'lower'